# Continuous Control

---

Congratulations for completing the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control an agent in a more challenging environment, where the goal is to train a creature with four arms to walk forward.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

from collections import deque
from d4pg_agent import Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Crawler.app"`
- **Windows** (x86): `"path/to/Crawler_Windows_x86/Crawler.exe"`
- **Windows** (x86_64): `"path/to/Crawler_Windows_x86_64/Crawler.exe"`
- **Linux** (x86): `"path/to/Crawler_Linux/Crawler.x86"`
- **Linux** (x86_64): `"path/to/Crawler_Linux/Crawler.x86_64"`
- **Linux** (x86, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86"`
- **Linux** (x86_64, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86_64"`

For instance, if you are using a Mac, then you downloaded `Crawler.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Crawler.app")
```

In [7]:
# env = UnityEnvironment(file_name='Crawler_Windows_x86_64/Crawler.exe')
env = UnityEnvironment(file_name='Crawler_Windows_x86_64/Crawler.exe', no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: CrawlerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 129
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 20
        Vector Action descriptions: , , , , , , , , , , , , , , , , , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [8]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [9]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 12
Size of each action: 20
There are 12 agents. Each observes a state with length: 129
The state for the first agent looks like: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.25000000e+00
  1.00000000e+00  0.00000000e+00  1.78813934e-07  0.00000000e+00
  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.06093168e-01 -1.42857209e-01 -6.06078804e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.33339906e+00 -1.42857209e-01
 -1.33341408e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -6.0609

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [10]:
# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [11]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [13]:
def ddpg(scores, n_episodes=1000, max_t=300, print_every=100, save_every=100, target_score=1000.0, N_step_bootstrap=5, prefill_memory_qty=0):
    
    # Abstract this into a seperate PARAM class
    GAMMA = 0.99
    
    env_info = env.reset(train_mode=True)[brain_name]
    NUM_ROBOTS = len(env_info.agents)
    scores_deque = deque(maxlen=print_every)
    nstep_exp_buffer = deque(maxlen=N_step_bootstrap)
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations              # get the current state
        agent.reset()
        score = 0
        nstep_exp_buffer.clear()
        
        for t in range(max_t):                             
            
            # Perform actions from each agent's policy network (clipped actions [0, -1])
            actions = [ agent.act(state) for state in states ]  
            actions = np.stack(actions)            
            env_info = env.step(actions)[brain_name]      
            next_states, rewards, dones = env_info.vector_observations, env_info.rewards, env_info.local_done
            
            # Train each agent using the same network + save experiences in replay buffer (w/ N-Step Bootstrap)
            experience = (states, actions, rewards)
            nstep_exp_buffer.append(experience)
            if len(nstep_exp_buffer) >= N_step_bootstrap:
                
                discounted_future_rewards = [0] * NUM_ROBOTS
                for i, exp in enumerate(nstep_exp_buffer):
                    discounted_future_rewards += ((GAMMA**i) * np.array(exp[2]))
#                     print("discounted_future_rewards: ", discounted_future_rewards)
                
                initial_exp = nstep_exp_buffer.popleft()
                for robot in range(NUM_ROBOTS):
                    agent.step(initial_exp[0][robot], initial_exp[1][robot], discounted_future_rewards[robot], next_states[robot], dones[robot], prefill_memory_qty)
                
            states = next_states
            score += (sum(rewards) / len(rewards))  # ALT METRIC: Max
            
            # Prevent agent from learning NaN rewards
            has_NAN_in_rewards = np.isnan(rewards)
            if any(has_NAN_in_rewards):
                rewards[has_NAN_in_rewards] = -5
                print("NaN Reward Found! Converting to R=-5 instead...")
            
            # Break if any of the agent is done
            if any(dones):
                break 
            
        # Print crucial results for progress tracking
        if agent.memory_buffer_prefilled(prefill_memory_qty):
            
            scores_deque.append(score)
            scores.append(score)
            print('\rEpisode {}\ Score: {:.2f}'.format(i_episode, scores_deque[-1]), end="")

            if i_episode % save_every == 0:
                torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
                torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')

            if i_episode % print_every == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

                if np.mean(scores_deque) >= target_score:
                    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'
                          .format(i_episode, np.mean(scores_deque)))
                    break

    return scores

############################## MAIN ##############################
print("CUDA AVAILABLE: ", torch.cuda.is_available())
scores = []
agent = Agent(state_size, action_size, random_seed=2)
ddpg(scores, n_episodes=10000, max_t=3000, print_every=100, save_every=100, target_score=1000, N_step_bootstrap=5, prefill_memory_qty=5000)  

CUDA AVAILABLE:  True
=============== NETWORKS ===============
actor_local Actor(
  (fc1): Linear(in_features=129, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=20, bias=True)
)
actor_target Actor(
  (fc1): Linear(in_features=129, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=20, bias=True)
)
critic_local Critic(
  (fc1): Linear(in_features=129, out_features=400, bias=True)
  (fc2): Linear(in_features=420, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=100, bias=True)
)
critic_target Critic(
  (fc1): Linear(in_features=129, out_features=400, bias=True)
  (fc2): Linear(in_features=420, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=100, bias=True)
)

=============== HYPERPARAMS ===============
DEVICE:  cuda:0
BUFFER_SIZE:  300000
BATCH_SIZE:  128
GAMMA:

KeyboardInterrupt: 

In [ ]:
# Plot a graph
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### RESUME TRAINING

Run this cell to load latest weights, just in case you want to resume training...

In [ ]:
# ############################## MAIN ##############################
# print("CUDA AVAILABLE: ", torch.cuda.is_available())
# scores = []
# agent = Agent(state_size, action_size, random_seed=2)
# agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
# agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))
# ddpg(scores, n_episodes=5000, max_t=1000, print_every=100, target_score=1000)  

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
# # load the weights from file
# agent = Agent(state_size, action_size, random_seed=1)
# agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
# agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))


# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment
# states = env_info.vector_observations                # get the current state
# score = 0                                              # initialize the score
# while True:
#     # Perform actions from each agent's policy network (clipped actions [0, -1])
#     actions = [ agent.act(state) for state in states ]  
#     actions = np.stack(actions)      
#     env_info = env.step(actions)[brain_name]  # send the action to the environment (BUG: Cast action to int)
#     next_states = env_info.vector_observations       # get the next state
#     rewards = env_info.rewards                       # get the reward
#     dones = env_info.local_done                      # see if episode has finished
#     score += max(rewards)                                    # update the score
#     states = next_states                                 # roll over the state to next time step
#     if max(dones):                                           # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
# env.close()